In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install torch transformers

In [16]:
!pip install langchain

In [9]:
!pip install accelerate

In [10]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [18]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00


In [19]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [12]:
base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [27]:
from transformers import pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
import sqlite3
import pandas as pd

# Setup the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=500,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)

# Wrap the pipeline with HuggingFacePipeline
local_llm = HuggingFacePipeline(pipeline=pipe)

# Define the template for SQL query generation
template = """
Write an SQL Query given the table name {Table} and columns as a list {Columns} for the given question:
{question}.
"""

# Setup the prompt template
prompt = PromptTemplate(template=template, input_variables=["Table", "question", "Columns"])

# Define a function to get the LLM response
def get_llm_response(tble, question, cols):
    llm_chain = LLMChain(prompt=prompt, llm=local_llm)
    response = llm_chain.run({"Table": tble, "question": question, "Columns": cols})
    return response.strip()

# Function to extract the SQL query from the response
def extract_sql_query(response):
    lines = response.split('\n')
    for line in lines:
        if line.strip().startswith('SELECT'):
            return line.strip()
    return None

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('my_database.db')

# Load data from CSV file into a DataFrame
df = pd.read_csv('/content/drive/My Drive/NL2SQL/dataset.csv')

# Write the data from pandas DataFrame to SQLite table
df.to_sql('images', conn, if_exists='replace', index=False)

# Example usage
tble = "images"
cols = ["img_id", "Country"]
question = "Give me the images of Japan"

# Generate the SQL query from the natural language question
response = get_llm_response(tble, question, cols)

# Extract the SQL query from the response
sql_query = extract_sql_query(response)

if sql_query:
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the generated SQL query
    cursor.execute(sql_query)

    # Fetch all rows of the query result
    results = cursor.fetchall()

    # Display the natural language question and the output image ids
    print(f"Natural Language Question: {question}")
    if results:
        print("Output Image IDs:")
        for row in results:
            print(row[0])
    else:
        print("No images found.")

    # Close the cursor and connection to free up resources
    cursor.close()
else:
    print("Failed to extract a valid SQL query from the response.")

conn.close()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Natural Language Question: Give me the images of Japan
Output Image IDs:
21.0
52.0
1091.0
1099.0
56.0
1097.0
